In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#  Project Objective
Investigate whether there is any measurable difference in progress based on the eligibility category. There are three options:
*Preliminary Analysis*
+ Developmental evaluation (delay)
+ Diagnosed condition
+ Diagnosed condition, developmental evaluation (by both delay and diagnosis)
1. Start by looking at overall progress by eligibility category (column AI of the "Elig Timeline Rpt 2018-2022" tab)
2. Factor in the time of service ("ECO with Exit-21-22" tab)
3. Do the above comparison by POE as well (column A of the "ECO with Exit21-22")
*Additional Analysis*
Additional areas you can look into are listed below:
+ Does typical time of service differ for different eligibility categories?


In [ ]:
teis = pd.ExcelFile("../Data/TEIS_NSS_Project_Data_10_2022.xlsx")
eco_21_22_exit = pd.read_excel(teis, 'ECO with Exit21-22', nrows=8633)
elig = pd.read_excel(teis, 'Elig Timeline Rpt 2018-2022', nrows=67611)

In [ ]:
eco_21_22_exit["CHILD_ID"] = eco_21_22_exit["CHILD_ID"].fillna(0).apply(np.int64)
eco_21_22_exit.info()

In [ ]:
elig = elig[['Child ID', 'Init. Elig. Category']]

In [ ]:
eco_21_22_exit = eco_21_22_exit[['CHILD_ID', 'ECO_Entry_DATE', 'ECO_Exit_DATE', 'Exit Date', 'Days btw Initial and Exit', 'Days btw I-IFSP to Exit ECO']]

In [ ]:
elig.rename(columns = {'Child ID':'CHILD_ID'}, inplace = True)

In [ ]:
eco_exit_elig = pd.merge(elig, eco_21_22_exit, how="inner", on=["CHILD_ID"])

In [ ]:
eco_exit_elig

Found total number of days

In [ ]:
eco_exit_elig['TOTAL DAYS IN ECO'] = (eco_exit_elig['ECO_Exit_DATE'] - eco_exit_elig['ECO_Entry_DATE']) / np.timedelta64(1, 'D')

In [ ]:
eco_exit_elig

In [ ]:
eco_exit_elig['ECO_ENTRY to Exit Date'] = (eco_exit_elig['Exit Date'] - eco_exit_elig['ECO_Entry_DATE']) / np.timedelta64(1, 'D')

Filtered out cases where total time was less than 6 months

In [ ]:
eco_exit_elig=eco_exit_elig[eco_exit_elig['TOTAL DAYS IN ECO']>=183]
eco_exit_elig=eco_exit_elig[eco_exit_elig['Days btw I-IFSP to Exit ECO']>=0]

In [ ]:
eco_exit_elig.info()

In [ ]:
avg_eco_exit_elig = eco_exit_elig.groupby('Init. Elig. Category', as_index=False)["Days btw Initial and Exit", "Days btw I-IFSP to Exit ECO", "TOTAL DAYS IN ECO", "ECO_ENTRY to Exit Date"].mean()
avg_eco_exit_elig

In [ ]:
avg_tidy = avg_eco_exit_elig.melt(id_vars='Init. Elig. Category', var_name = 'Days Between')
avg_tidy


In [ ]:
avg_tidy1 = avg_tidy.replace(regex={'Developmental Evaluation': 'D.V.', 'Diagnosed Condition': 'D.C.'})
avg_tidy1

In [ ]:
sns.barplot(data=avg_tidy1, x="Days Between", y="value", hue="Init. Elig. Category")
plt.tick_params(axis='both', which='major', labelsize=15)
sns.set(rc = {'figure.figsize':(15, 5)})

In [ ]:
avg_eco = sns.FacetGrid(avg_tidy1, col = 'Days Between')
avg_eco.map(sns.barplot, "Init. Elig. Category", "value")
avg_eco.set_titles(col_template="{col_name}", row_template="{row_name}")